<a href="https://colab.research.google.com/github/vadigr123/colab-testing/blob/main/video_to_frames_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@markdown ### 🖼️ Choose frame image format
frame_format = "png"  #@param ["png", "jpg", "jpeg", "bmp", "tiff", "webp"] {type:"string"}

#@markdown ### 📦 Upload a ZIP file containing video files (.mp4, .webm, etc.)
from google.colab import files
import os
import zipfile
import shutil
import glob

# Upload ZIP file
uploaded = files.upload()
zip_path = list(uploaded.keys())[0]

# Extract the uploaded ZIP
extract_dir = "videos"
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"✅ ZIP extracted!")

# Frame output directory
frames_root = "frames_output"
os.makedirs(frames_root, exist_ok=True)

# Supported video formats
supported_exts = [".mp4", ".webm", ".mov", ".mkv", ".avi"]

# Find video files
video_files = glob.glob(f"{extract_dir}/*")
video_files = [f for f in video_files if os.path.splitext(f)[1].lower() in supported_exts]

if not video_files:
    print("⚠️ No supported video files found.")
else:
    for video_file in video_files:
        filename = os.path.splitext(os.path.basename(video_file))[0]
        out_folder = os.path.join(frames_root, filename)
        os.makedirs(out_folder, exist_ok=True)

        print(f"🎞️ Extracting frames from {filename}...")

        # FFmpeg command to extract frames
        !ffmpeg -i "{video_file}" "{out_folder}/frame_%04d.{frame_format}" -hide_banner -loglevel error

# Create final ZIP with all frame folders
final_zip = "extracted-frames.zip"
shutil.make_archive("extracted-frames", 'zip', frames_root)

# Download final zip
files.download(final_zip)

print("✅ Done!")


Saving Fumrryms.zip to Fumrryms.zip
✅ ZIP extracted!
🎞️ Extracting frames from 013_wolf...
🎞️ Extracting frames from 043_cat_face...
🎞️ Extracting frames from 010_fox...
🎞️ Extracting frames from 042_wolf...
🎞️ Extracting frames from 028_lion...
🎞️ Extracting frames from 012_wolf...
🎞️ Extracting frames from 019_wolf...
🎞️ Extracting frames from 036_otter...
🎞️ Extracting frames from 025_crocodile...
🎞️ Extracting frames from 021_wolf...
🎞️ Extracting frames from 044_dragon_face...
🎞️ Extracting frames from 016_cheese_wedge...
🎞️ Extracting frames from 009_wolf...
🎞️ Extracting frames from 000_fox...
🎞️ Extracting frames from 015_sparkles...
🎞️ Extracting frames from 017_wolf...
🎞️ Extracting frames from 005_wolf...
🎞️ Extracting frames from 018_wolf...
🎞️ Extracting frames from 032_dragon_face...
🎞️ Extracting frames from 040_deer...
🎞️ Extracting frames from 030_wolf...
🎞️ Extracting frames from 001_wolf...
🎞️ Extracting frames from 007_wolf...
🎞️ Extracting frames from 039_raccoon..

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Done!
